In [222]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
from typing import List, Tuple, Any, Dict
from mcbo.optimizers.bo_builder import BoBuilder
from mcbo.tasks.task_base import TaskBase
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()


df = pd.read_csv('train.csv')
print(df.shape)
df = df.dropna(subset=['Age', 'Fare', 'Sex'])
target_var = df.Survived
print(df.shape)

df = df[['Sex','Age', 'Fare']]
encode_cat_var = pd.get_dummies(df['Sex'])
df = df.drop(['Sex'], axis =1)
X = pd.concat([encode_cat_var, df], axis = 1)
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns) 

y = target_var
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Activation functions
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Loss function
def binary_cross_entropy(y_true, y_pred):
    '''epsilon = 1e-15  # Small value to avoid numerical instability
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip predicted probabilities to avoid taking the log of zero or one
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
'''
    cost = np.mean(np.abs(y_true - y_pred))  # Example: Mean absolute error
    return cost

# Feed-forward ANN
def forward_prop(W1, b1, W2, b2, X_data):
    n_features = X_data.shape[1]
    if W1.shape[1] != n_features:
        W1_shape_0 = W1.size / n_features 
        W1 = np.reshape(W1_shape_0 ,n_features)
    if W2.shape[1] != W1.shape[0]:
        W2_shape_0 = W2.size / W1.shape[0]
        W2 = np.reshape((W2_shape_0, W1.shape[0]))
    Z1 = np.dot(W1, X_data.T) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2).T 
    return A2


# Evaluation function
def black_box_function(W1, b1, W2, b2):
    result = forward_prop(W1, b1, W2, b2, X_train)
    y_train_encoded = np.array([y_train]) # np.eye(2)[y_train]
    
    return binary_cross_entropy(y_train_encoded, result)

class CustomTask(TaskBase):
    @property
    def name(self) -> str:
        return 'Custom Task'

    def evaluate(self, x: np.ndarray) -> np.ndarray:
        y = np.zeros((len(x), 1))
        print(x, '--------------------------------------------')
        for ind in range(len(x)):
            x_ind = x.iloc[ind].to_numpy()  # Convert Series to NumPy array
            print("Evaluating with parameters:")
            print("x_ind:", x_ind)
            print('xind shape' , x_ind.shape)
            y[ind] = black_box_function(x_ind[:16].reshape(4, 4), x_ind[16:20].reshape(4, 1), 
                                        x_ind[20:24].reshape(1, 4), x_ind[24:].reshape(1,1))
        return y

    
    def get_search_space_params(self) -> List[Dict[str, Any]]:
        params = [{'name': f'W1_{i}{j}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(4) for j in range(4)]
        params.extend([{'name': f'b1_{i}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(4)])
        params.extend([{'name': f'W2_{i}{j}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(1) for j in range(4)])
        params.extend([{'name': f'b2_{i}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(1)])
        return params

    def get_parameter_names(self) -> List[str]:
        params = []
        for i in range(4):
            for j in range(4):
                params.append(f'W1_{i}{j}')
        params.extend([f'b1_{i}' for i in range(4)])
        for i in range(1):
            for j in range(4):
                params.append(f'W2_{i}{j}')
        params.extend([f'b2_{i}' for i in range(1)])
        return params

# Creating task and optimization
task = CustomTask()
searchspace = task.get_search_space()

epochs = 20
bo_builder = BoBuilder(model_id='gp_hed', acq_opt_id='is', acq_func_id='ei', tr_id='basic')
opt = bo_builder.build_bo(search_space=task.get_search_space(), n_init=50)

# Optimization loop
budget_eval = epochs
# Inside the optimization loop
for _ in range(budget_eval):
    x_next = opt.suggest()
    x_next_reshaped = np.array(x_next).reshape(1, -1)  # Reshape x_next to have shape (1, 25)
    x_next_df = pd.DataFrame(x_next_reshaped, columns=task.get_parameter_names())
    print(x_next_df.shape, 'xnextdf')
    y_next = task.evaluate(x_next_df)
    opt.observe(x_next, y_next)

    
# Printing best result
print('Best parameters:', opt.best_x)
print('Best loss:', opt.best_y)

# Plotting version
fig, ax = plt.subplots(figsize=(7, 5))
y = opt.data_buffer.y.numpy()
regret_y = np.min(np.cumsum(y, axis=1), axis=0)
#plt.plot(np.arange(1,  budget_eval ), regret_y)
#plt.show()

(891, 12)
(714, 12)
(1, 25) xnextdf
      W1_00     W1_01     W1_02     W1_03     W1_10     W1_11     W1_12  \
0  0.694593 -0.075086 -0.016972 -0.506505  0.531527 -0.352209 -0.059352   

      W1_13     W1_20     W1_21  ...     W1_33      b1_0      b1_1      b1_2  \
0  0.617955  0.170972  0.964155  ...  0.439117 -0.723536  0.104445  0.441786   

       b1_3     W2_00     W2_01     W2_02     W2_03      b2_0  
0 -0.265652  0.464602 -0.577474  0.753196 -0.407304  0.247507  

[1 rows x 25 columns] --------------------------------------------
Evaluating with parameters:
x_ind: [ 0.69459349 -0.07508637 -0.01697155 -0.50650516  0.53152664 -0.35220889
 -0.0593519   0.6179555   0.17097188  0.96415476 -0.05648738 -0.91623921
  0.37940307  0.28710425  0.39301437  0.43911683 -0.72353583  0.10444456
  0.44178644 -0.26565237  0.46460154 -0.57747417  0.75319624 -0.40730385
  0.24750677]
xind shape (25,)
(1, 25) xnextdf
      W1_00     W1_01     W1_02     W1_03     W1_10     W1_11     W1_12  \
0 -0.11

(1, 25) xnextdf
     W1_00     W1_01     W1_02     W1_03     W1_10     W1_11     W1_12  \
0  0.68288  0.505972  0.408209  0.378218  0.955987  0.027267  0.085683   

      W1_13     W1_20     W1_21  ...    W1_33      b1_0      b1_1      b1_2  \
0 -0.101704  0.211519  0.009479  ... -0.93996  0.744212 -0.177906 -0.813425   

       b1_3     W2_00     W2_01     W2_02    W2_03      b2_0  
0  0.829772  0.804491  0.996842 -0.983784 -0.91086 -0.071455  

[1 rows x 25 columns] --------------------------------------------
Evaluating with parameters:
x_ind: [ 0.68287952  0.50597239  0.40820872  0.37821816  0.95598685  0.02726677
  0.08568275 -0.10170359  0.21151898  0.00947895 -0.19290276  0.46490926
 -0.71991038  0.72787161  0.02985899 -0.93995955  0.74421213 -0.17790561
 -0.81342523  0.8297723   0.80449125  0.99684247 -0.98378361 -0.91085992
 -0.07145526]
xind shape (25,)
(1, 25) xnextdf
      W1_00     W1_01     W1_02     W1_03     W1_10     W1_11     W1_12  \
0  0.235389  0.622325  0.179458  

In [223]:
l1=np.array([-0.61717523  ,0.36990035,  0.49814788, -0.5190669,   0.51623886 ,-0.35973204
 , 0.1933757  , 0.67851127 , 0.19358574, -0.43996284, -0.62441788 , 0.3295846
, -0.4392478 , -0.55685931 ,-0.26148563 , 0.62263243 , 0.84988384, -0.54001019
, -0.4897618  ,-0.74264208, -0.32725616, -0.69292137 ,-0.18909265,  0.39931873
,  0.57353324])

par1,par2,par3,par4= l1[:16].reshape(4, 4), l1[16:20].reshape(4, 1), l1[20:24].reshape(1, 4), l1[24:].reshape(1,1)
for_prop=(forward_prop(par1,par2,par3,par4, X_train))
y_train_encoded = np.array([y_train]) # np.eye(2)[y_train]
res = binary_cross_entropy( y_train_encoded, for_prop)
print(res)

0.5106061662116507


In [224]:
dict_weights_and_biases = dict(opt.best_x)
opt.best_x

,W1_00,W1_01,W1_02,W1_03,W1_10,W1_11,W1_12,W1_13,W1_20,W1_21,...,W1_33,b1_0,b1_1,b1_2,b1_3,W2_00,W2_01,W2_02,W2_03,b2_0
0,0.657061,0.293242,0.913671,0.571333,-0.36779,-0.80862,0.310766,-0.572527,0.146803,-0.797944,...,0.033472,-0.243607,0.08836,0.141123,-0.38469,-0.803011,0.71257,-0.954861,-0.362605,-0.57053


In [225]:
def weights_and_biases_matrix(df_pars):
    # Group the columns by their prefixes
    grouped_columns = df_pars.columns.to_series().groupby(lambda x: x.split('_')[0], sort=False)

    # Use dictionary comprehension to create the result dictionary
   # result_arrays = {prefix: np.vstack([df_pars[cols].columns, df_pars[cols].values])
    #                 for prefix, cols in grouped_columns.groups.items()}
    result_arrays = {prefix: np.array(df_pars[cols].values)
                     for prefix, cols in grouped_columns.groups.items()}

    return result_arrays

    #return weights_set_list#W1, weights_W2

weights_and_biases_results = (weights_and_biases_matrix(opt.best_x))
(weights_and_biases_results)


{'W1': array([[ 0.65706052,  0.29324227,  0.91367134,  0.5713332 , -0.36779022,
         -0.80861977,  0.3107664 , -0.57252662,  0.14680325, -0.79794351,
         -0.04675909, -0.91023797,  0.8867146 ,  0.1539183 ,  0.86388828,
          0.03347209]]),
 'b1': array([[-0.2436073 ,  0.08836023,  0.14112281, -0.38468983]]),
 'W2': array([[-0.80301072,  0.71257026, -0.95486061, -0.36260549]]),
 'b2': array([[-0.57052975]])}

In [226]:
def forward_prop_adj(W1, b1, W2, b2, X_data):
    n_features = X_data.shape[1]
    if W1.shape[1] != n_features:
        W1_shape_0 = int(W1.size / n_features) 
        W1 = np.reshape(W1, (W1_shape_0 ,n_features))
    if W2.shape[1] != W1.shape[0]:
        W2_shape_0 = int(W2.size / W1.shape[0])
        W2 = np.reshape(W2, (W2_shape_0, W1.shape[0]))
    Z1 = np.dot(W1, X_data.T) + b1.T
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2.T
    A2 = sigmoid(Z2).T 
    return A2

def binarize_outcome(res_arr):
    return np.where(res_arr<.5,0,1)

def calculate_accuracy(actual_val, pred_val):
    correct = np.sum(actual_val == pred_val)
    total = len(actual_val)
    accuracy = correct / total
    print(correct)
    print(total)
    return accuracy

preds_train_bo = forward_prop_adj(**weights_and_biases_results, X_data=X_train)
binary_res = binarize_outcome(preds_train_bo)
print('accuracy =', calculate_accuracy(np.array(y_train), binary_res.flatten()))

287
478
accuracy = 0.600418410041841


In [227]:
from sklearn import metrics
print('accuracy =', calculate_accuracy(np.array(y_train), binary_res.flatten()))
confusion_matrix = metrics.confusion_matrix(y_train, binary_res.flatten())
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()


287
478
accuracy = 0.600418410041841


In [229]:
print(binary_res.flatten())
plt.plot(preds_train_bo)
plt.show()

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [143]:
print(preds_train_bo)
plt.hist(preds_train_bo)
plt.show()

[[0.40761507]
 [0.470208  ]
 [0.40554545]
 [0.470208  ]
 [0.470208  ]
 [0.41248621]
 [0.44916972]
 [0.41230894]
 [0.40797339]
 [0.40860949]
 [0.470208  ]
 [0.470208  ]
 [0.40916443]
 [0.470208  ]
 [0.40814292]
 [0.470208  ]
 [0.41204252]
 [0.40908308]
 [0.40793348]
 [0.470208  ]
 [0.470208  ]
 [0.40881179]
 [0.470208  ]
 [0.470208  ]
 [0.470208  ]
 [0.470208  ]
 [0.470208  ]
 [0.40520101]
 [0.40740696]
 [0.40725289]
 [0.40419544]
 [0.470208  ]
 [0.43576029]
 [0.470208  ]
 [0.40754705]
 [0.470208  ]
 [0.40927395]
 [0.40502724]
 [0.4110313 ]
 [0.470208  ]
 [0.4114367 ]
 [0.470208  ]
 [0.40688346]
 [0.40819908]
 [0.470208  ]
 [0.470208  ]
 [0.40541291]
 [0.40687646]
 [0.470208  ]
 [0.40918978]
 [0.40778645]
 [0.40760307]
 [0.40805274]
 [0.40819044]
 [0.40841989]
 [0.40699564]
 [0.40805086]
 [0.4045623 ]
 [0.41065184]
 [0.470208  ]
 [0.41047419]
 [0.40886334]
 [0.4118736 ]
 [0.470208  ]
 [0.47128252]
 [0.470208  ]
 [0.40615556]
 [0.470208  ]
 [0.4085976 ]
 [0.40648548]
 [0.40860949]
 [0.40

In [124]:
import numpy as np

# Define the Eggholder function
def eggholder(x, y):
    return (-(y + 47) * np.sin(np.sqrt(np.abs(x / 2 + (y + 47)))) - x * np.sin(np.sqrt(np.abs(x - (y + 47)))))

# Define the range of x and y values
x_min, x_max = -512, 512
y_min, y_max = -512, 512

# Define the number of points per dimension
num_points_per_dimension =25

# Generate X1, X2, X3, and X4 coordinates with the specified number of points
x = np.linspace(x_min, x_max, num_points_per_dimension)
y = np.linspace(y_min, y_max, num_points_per_dimension)

# Generate the meshgrid
X, Y = np.meshgrid(x, y)

# Calculate Z values using the Eggholder function
Z_eggholder = eggholder(X, Y)

# Flatten X, Y, and Z to create the predictors
X1_flat = X.flatten()
X2_flat = Y.flatten()
X3_flat = (X ** 2).flatten()
X4_flat = (Y ** 2).flatten()

# Combine X1, X2, X3, and X4 to create the predictors
X = np.column_stack((X1_flat, X2_flat, X3_flat, X4_flat))

# Binarize the Z_eggholder values to create the binary target variable
threshold = np.mean(Z_eggholder)
y = (Z_eggholder.flatten() > threshold).astype(int)

# Display the shapes of predictors and target
print("Shape of predictors:", X.shape)
print("Shape of target:", y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


Shape of predictors: (625, 4)
Shape of target: (625,)


In [125]:
import numpy as np

def binarize_outcome(res_arr):
    return np.where(res_arr<.5,0,1)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

def initialize_parameters(input_size, hidden_size, output_size):
    np.random.seed(0)
    W1 = np.random.randn(hidden_size, input_size) * 0.01
    b1 = np.zeros((hidden_size, 1))
    W2 = np.random.randn(output_size, hidden_size) * 0.01
    b2 = np.zeros((output_size, 1))
    return {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}

def forward_propagation(X, parameters):
    W1, b1, W2, b2 = parameters['W1'], parameters['b1'], parameters['W2'], parameters['b2']
    Z1 = np.dot(W1, X.T) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    cache = {'Z1': Z1, 'A1': A1, 'Z2': Z2, 'A2': A2}
    return A2, cache
    
def compute_cost(A2, Y):
    m = 1
    cost = -np.sum(Y * np.log(A2) + (1 - Y) * np.log(1 - A2)) / m
    return np.squeeze(cost)

def backward_propagation(X, Y, parameters, cache):
    m = X.shape[1]
    A1, A2 = cache['A1'], cache['A2']
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m
    dZ1 = np.dot(parameters['W2'].T, dZ2) * relu_derivative(A1)
    dW1 = np.dot(dZ1, X) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m
    return {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2}

def update_parameters(parameters, grads, learning_rate):
    parameters['W1'] -= learning_rate * grads['dW1']
    parameters['b1'] -= learning_rate * grads['db1']
    parameters['W2'] -= learning_rate * grads['dW2']
    parameters['b2'] -= learning_rate * grads['db2']
    return parameters

def train_neural_network(X, Y, input_size, hidden_size, output_size, num_iterations=50, learning_rate=0.1):
    parameters = initialize_parameters(input_size, hidden_size, output_size)
    for i in range(num_iterations):
        A2, cache = forward_propagation(X, parameters)
        
        cost = compute_cost(A2, Y)
        grads = backward_propagation(X, Y, parameters, cache)
        parameters = update_parameters(parameters, grads, learning_rate)
        if i % 100 == 0:
            print(f'Cost after iteration {i}: {cost}')
            print(f'Output after iteration {i}: {A2}')
    return parameters, A2  # Return the trained parameters and final predictions

# Example usage:
# Generate random input and output data



# Train the neural network
input_size = 4
hidden_size = 4
output_size = 1
trained_parameters, final_predictions = train_neural_network(X_train, np.array(y_train), input_size, hidden_size, output_size, num_iterations=1000, learning_rate=0.01)

print("Final predictions:")
print(final_predictions)


C:\Users\ydg88\AppData\Local\Temp\ipykernel_6264\2329022958.py:35: RuntimeWarning: divide by zero encountered in log
  cost = -np.sum(Y * np.log(A2) + (1 - Y) * np.log(1 - A2)) / m
C:\Users\ydg88\AppData\Local\Temp\ipykernel_6264\2329022958.py:35: RuntimeWarning: invalid value encountered in multiply
  cost = -np.sum(Y * np.log(A2) + (1 - Y) * np.log(1 - A2)) / m


Cost after iteration 0: nan
Output after iteration 0: [[1.         1.         1.         0.96755257 0.9974931  1.
  1.         1.         1.         1.         1.         0.96340833
  0.99896578 0.99999991 1.         1.         0.7917838  1.
  1.         0.65426579 1.         1.         0.99999948 1.
  1.         1.         1.         0.96390026 0.99998404 1.
  0.99999999 1.         0.99998957 0.9993343  0.99999941 1.
  1.         1.         0.99999995 0.99999996 1.         1.
  0.99999998 1.         1.         1.         1.         0.99999995
  1.         0.93965784 1.         1.         1.         1.
  1.         1.         1.         1.         1.         0.99999294
  1.         1.         0.99999996 0.99999995 1.         1.
  1.         1.         1.         1.         1.         1.
  1.         0.99986656 1.         1.         0.99998072 0.99880273
  1.         0.99999998 0.99999948 1.         1.         1.
  1.         1.         1.         0.99982811 0.69556799 1.
  1.         0

C:\Users\ydg88\AppData\Local\Temp\ipykernel_6264\2329022958.py:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Cost after iteration 300: 289.69245779422783
Output after iteration 300: [[0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.507177

Cost after iteration 700: 289.69245779422783
Output after iteration 700: [[0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.507177

In [126]:
trained_parameters, final_predictions_train = train_neural_network(X_train, np.array(y_train), input_size, hidden_size, output_size, num_iterations=1000, learning_rate=0.01)

# Now, use the trained parameters to make predictions on the test data
final_predictions_test, _ = forward_propagation(X_test, trained_parameters)

print("Final predictions on training data:")
print(final_predictions_train)

print("Final predictions on test data:")
print(final_predictions_test)

C:\Users\ydg88\AppData\Local\Temp\ipykernel_6264\2329022958.py:35: RuntimeWarning: divide by zero encountered in log
  cost = -np.sum(Y * np.log(A2) + (1 - Y) * np.log(1 - A2)) / m
C:\Users\ydg88\AppData\Local\Temp\ipykernel_6264\2329022958.py:35: RuntimeWarning: invalid value encountered in multiply
  cost = -np.sum(Y * np.log(A2) + (1 - Y) * np.log(1 - A2)) / m
C:\Users\ydg88\AppData\Local\Temp\ipykernel_6264\2329022958.py:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Cost after iteration 0: nan
Output after iteration 0: [[1.         1.         1.         0.96755257 0.9974931  1.
  1.         1.         1.         1.         1.         0.96340833
  0.99896578 0.99999991 1.         1.         0.7917838  1.
  1.         0.65426579 1.         1.         0.99999948 1.
  1.         1.         1.         0.96390026 0.99998404 1.
  0.99999999 1.         0.99998957 0.9993343  0.99999941 1.
  1.         1.         0.99999995 0.99999996 1.         1.
  0.99999998 1.         1.         1.         1.         0.99999995
  1.         0.93965784 1.         1.         1.         1.
  1.         1.         1.         1.         1.         0.99999294
  1.         1.         0.99999996 0.99999995 1.         1.
  1.         1.         1.         1.         1.         1.
  1.         0.99986656 1.         1.         0.99998072 0.99880273
  1.         0.99999998 0.99999948 1.         1.         1.
  1.         1.         1.         0.99982811 0.69556799 1.
  1.         0

Cost after iteration 600: 289.69245779422783
Output after iteration 600: [[0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.50717703 0.50717703 0.50717703
  0.50717703 0.50717703 0.50717703 0.507177

In [127]:
def binarize_outcome(res_arr):
    return np.where(res_arr<.5,0,1)

def calculate_accuracy(actual_val, pred_val):
    correct = np.sum(actual_val == pred_val)
    total = len(actual_val)
    accuracy = correct / total
    print(correct)
    print(total)
    return accuracy


bin_out_backprop_train=(binarize_outcome(final_predictions_train[-1]))
print(calculate_accuracy(bin_out_backprop_train, y_train))
bin_out_backprop_test=(binarize_outcome(final_predictions_test[-1]))
print(calculate_accuracy(bin_out_backprop_test, y_test))


212
418
0.507177033492823
104
207
0.5024154589371981


In [123]:
y_test

array([0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1])

In [147]:
print((trained_parameters.values()))

dict_values([array([[ 0.88596038, -1.00956549,  1.22039933,  1.42074376],
       [ 0.52220002, -0.5955235 ,  0.71281002,  0.82938996],
       [-0.04633879,  0.03022445,  0.08010684,  0.0266258 ],
       [ 0.12226227, -0.00692515,  1.2344951 , -4.40113791]]), array([[-0.1452472 ],
       [-0.08222628],
       [-0.01918814],
       [ 0.10650999]]), array([[ 1.64631059,  0.96568291, -0.08349684, -2.36681665]]), array([[-0.56869235]])])


In [148]:
plt.scatter(np.concatenate(list(weights_and_biases_results.values()), axis = None),
np.concatenate(list(trained_parameters.values()), axis=None))
plt.show()

In [149]:
np.concatenate(list(weights_and_biases_results.values()), axis=None)

array([-0.63615757,  0.02715206, -0.14785868,  0.13741609,  0.24334712,
        0.31646704, -0.55240617,  0.54538903,  0.30573323, -0.1200694 ,
       -0.90739278,  0.91180403, -0.3979356 ,  0.36028264,  0.43749272,
       -0.65571392,  0.15265377, -0.35503987, -0.91208553,  0.04449118,
       -0.63336928,  0.93814317,  0.44029527, -0.31406779, -0.11930932])

In [150]:
plt.hist(final_predictions[-1])
plt.show()

In [234]:
import numpy as np

# Load your dataset here (replace this with your actual dataset)
# For demonstration, let's create a random dataset
np.random.seed(0)
X = np.array(X_train) #np.random.rand(428, 4)  # Predictor variables
y = np.array(y_train) #np.random.randint(2, size=428)  # Binary target variable (0 or 1)
print(X)
# Objective function (e.g., accuracy of a model)
def objective_function(x):
    coefficients = x 
    y_pred = np.dot(X, coefficients) 
    y_pred_binary = (y_pred >= 0.5).astype(int) 
    accuracy = np.mean(y_pred_binary == y)  # Calculate accuracy
    return accuracy, y_pred_binary

# Define search space (x_range) for each predictor variable
# You would replace these ranges with your actual ranges based on your data and domain knowledge
x1_range = [0, 1]  # Example range for predictor variable 1
x2_range = [0, 1]  # Example range for predictor variable 2
x3_range = [0, 1]  # Example range for predictor variable 3
x4_range = [0, 1]  # Example range for predictor variable 4

# Combine ranges into a single search space
x_range = [x1_range, x2_range, x3_range, x4_range]

# Bayesian optimization
def bayesian_optimization(objective_function_0, x_range, num_iterations):
    best_hyperparameters = None
    best_performance = -float('inf')
    
    for _ in range(num_iterations):
        # Randomly sample hyperparameters from the search space
        x_sample = [np.random.uniform(low=x_range[i][0], high=x_range[i][1]) for i in range(len(x_range))]
        print(x_sample, 'x_sample')
        # Evaluate objective function with the sampled hyperparameters
        performance = objective_function_0(x_sample)[0]
        predicted_values = objective_function_0(x_sample)[1]
        print(performance, 'performance')
        # Update best hyperparameters if performance improves
        if performance > best_performance:
            best_hyperparameters = x_sample
            best_performance = performance
            predicted_vals = predicted_values
    
    return best_hyperparameters, best_performance, predicted_vals

# Example usage
num_iterations = 100

best_hyperparameters, best_performance, pred_y =  bayesian_optimization(objective_function, x_range, num_iterations)
print("Best Hyperparameters:", best_hyperparameters)
print("Best Performance (Accuracy):", best_performance)
print('pred_y', pred_y)



[[0.         1.         0.30887158 0.        ]
 [1.         0.         0.19577783 0.07690368]
 [0.         1.         0.54762503 0.01571255]
 ...
 [1.         0.         0.50992712 0.26252652]
 [0.         1.         0.40939935 0.04006213]
 [0.         1.         0.40939935 0.01541158]]
[0.5488135039273248, 0.7151893663724195, 0.6027633760716439, 0.5448831829968969] x_sample
0.399581589958159 performance
[0.4236547993389047, 0.6458941130666561, 0.4375872112626925, 0.8917730007820798] x_sample
0.38493723849372385 performance
[0.9636627605010293, 0.3834415188257777, 0.7917250380826646, 0.5288949197529045] x_sample
0.39330543933054396 performance
[0.5680445610939323, 0.925596638292661, 0.07103605819788694, 0.08712929970154071] x_sample
0.399581589958159 performance
[0.02021839744032572, 0.832619845547938, 0.7781567509498505, 0.8700121482468192] x_sample
0.2824267782426778 performance
[0.978618342232764, 0.7991585642167236, 0.46147936225293185, 0.7805291762864555] x_sample
0.39958158995815

[0.09784448449403405, 0.8621915174216833, 0.9729194890231303, 0.9608346580630002] x_sample
0.3284518828451883 performance
[0.906555499221179, 0.7740473326986388, 0.3331451520286419, 0.08110138998799676] x_sample
0.399581589958159 performance
[0.40724117141380733, 0.23223414217094274, 0.13248763475798297, 0.05342718178682526] x_sample
0.6276150627615062 performance
[0.7255943642105788, 0.011427458625031028, 0.7705807485027762, 0.14694664540037505] x_sample
0.7301255230125523 performance
[0.07952208258675575, 0.08960303423860538, 0.6720478073539145, 0.24536720985284477] x_sample
0.5669456066945606 performance
[0.42053946668009845, 0.5573687913239169, 0.8605511738287938, 0.7270442627113283] x_sample
0.38493723849372385 performance
[0.27032790523871464, 0.1314827992911276, 0.05537432042119794, 0.3015986344809425] x_sample
0.600418410041841 performance
[0.26211814923967824, 0.45614056680047965, 0.6832813355476804, 0.6956254456388572] x_sample
0.3472803347280335 performance
[0.28351884658216

In [179]:
confusion_matrix = metrics.confusion_matrix(y_train, pred_y)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()


In [180]:
print(calculate_accuracy(y_train,pred_y))

379
478
0.7928870292887029


In [203]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Any, Dict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load your dataset here (replace this with your actual dataset)
df = pd.read_csv('train.csv').dropna(subset=['Age', 'Fare', 'Sex'])
target_var = df.Survived
df = df[['Sex', 'Age', 'Fare']]
encode_cat_var = pd.get_dummies(df['Sex'])
df = df.drop(['Sex'], axis=1)
X = pd.concat([encode_cat_var, df], axis=1)
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
y = target_var.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Activation functions
def relu(Z):
    return np.maximum(0, Z)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Loss function
def binary_cross_entropy(y_true, y_pred):
    cost = np.mean(np.abs(y_true - y_pred))  # Example: Mean absolute error
    return cost

# Feed-forward ANN
def forward_prop(W1, b1, W2, b2, X_data):
    Z1 = np.dot(X_data, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = sigmoid(Z2)
    return A2

# Evaluation function
def black_box_function(W1, b1, W2, b2):
    result = forward_prop(W1, b1, W2, b2, X_train)
    return binary_cross_entropy(y_train, result)

# Define search space parameters
def get_search_space_params() -> List[Dict[str, Any]]:
    params = [{'name': f'W1_{i}{j}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(4) for j in range(4)]
    params.extend([{'name': f'b1_{i}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(4)])
    params.extend([{'name': f'W2_{i}{j}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(1) for j in range(4)])
    params.extend([{'name': f'b2_{i}', 'type': 'num', 'lb': -1, 'ub': 1} for i in range(1)])
    return params

# Creating task and optimization
def optimize(epochs):
    np.random.seed(42)
    search_space_params = get_search_space_params()
    best_loss = float('inf')
    best_params = None
    for _ in range(epochs):
        params = [np.random.uniform(p['lb'], p['ub']) for p in search_space_params]
        W1 = np.array(params[:16]).reshape(4, 4)
        b1 = np.array(params[16:20]).reshape(1, 4)
        W2 = np.array(params[20:24]).reshape(4, 1)
        b2 = np.array(params[24]).reshape(1, 1)
        loss = black_box_function(W1, b1, W2, b2)
        if loss < best_loss:
            best_loss = loss
            best_params = params
    return best_params, best_loss

# Example usage
epochs = 20
best_params, best_loss = optimize(epochs)
print('Best parameters:', best_params)
print('Best loss:', best_loss)


Best parameters: [0.006272517160175406, 0.7129796823766446, 0.31738726323789, -0.6741311458371406, -0.8588625051991403, 0.2848385564126312, -0.9469773789167564, 0.1715511625469266, 0.8804604828499152, 0.1509483557517579, -0.22366014758695618, 0.28657643688470635, -0.08349421901696674, 0.09123357863186987, 0.8829296175530503, -0.2277947243984515, 0.9223811276478284, 0.8107012839121275, -0.6084177304214071, -0.8612773982496691, -0.7984439972451467, -0.9635563486969005, -0.8111140784881432, 0.3660135468327137, -0.857622703079542]
Best loss: 0.4138790781738342


In [1]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
from typing import List, Tuple, Any, Dict
from mcbo.optimizers.bo_builder import BoBuilder
from mcbo.tasks.task_base import TaskBase
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()


df = pd.read_csv('train.csv')
print(df.shape)
df = df.dropna(subset=['Age', 'Fare', 'Sex'])
target_var = df.Survived
print(df.shape)

df = df[['Sex','Age', 'Fare']]
encode_cat_var = pd.get_dummies(df['Sex'])
df = df.drop(['Sex'], axis =1)
X = pd.concat([encode_cat_var, df], axis = 1)
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns) 

y = target_var
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



def label_parameters(layer_sizes):
    labels = {}
    for i in range(len(layer_sizes) - 1):
        labels[f"W{i}"] = f"{i}"
        labels[f"b{i}"] = f"{i}"
    return labels

class FeedForwardNN:
    def __init__(self, parameter_labels, X_data):
        self.parameter_labels = parameter_labels
        self.parameters = {}
        for param, label in self.parameter_labels:             #.items():
            self.parameters[param] = None  # Initialize parameters to None

    def initialize_parameters(self):
        for param in self.parameter_labels.keys():
            # Initialize parameters randomly or with zeros, depending on your preference
            self.parameters[param] = np.random.randn(3, 3)  # Example initialization

    def forward_propagation(self, X_data):
        # Implement forward propagation using parameters stored in self.parameters
        pass  # Placeholder for implementation

  #  def train(self, X_train, y_train):
        # Placeholder for training process
   #     pass

# Example usage:
layer_sizes = np.array([4, 5, 3, 1])  # Example list of layer sizes
parameter_labels = label_parameters(layer_sizes)  # Using previously defined label_parameters function
#nn = FeedForwardNN(parameter_labels)
#nn.initialize_parameters()  # Initialize parameters
#activations = nn.forward_propagation(X_train)
parameters =np.array(list( parameter_labels.keys()))
print((parameters))





# Activation functions
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Loss function
def binary_cross_entropy(y_true, y_pred):
    '''epsilon = 1e-15  # Small value to avoid numerical instability
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip predicted probabilities to avoid taking the log of zero or one
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
'''
    cost = np.mean(np.abs(y_true - y_pred))  # Example: Mean absolute error
    return cost

def generate_sizes(layer_sizes):
    sizes = []
    start_idx = 0
    for size in layer_sizes:
        end_idx = start_idx + size
        sizes.append(end_idx - start_idx)
        start_idx = end_idx
    return sizes


def black_box_function(*args, layer_sizes):
    size_tuples = [(layer_sizes[i], layer_sizes[i+1]) for i in range(len(layer_sizes)-1)]
    arr_args = [np.array(arg) for arg in args]  # Convert each argument to a NumPy array
    print(arr_args, 'arr args')
    reshaped_args = [arg.reshape(size) for arg, size in zip(arr_args, size_tuples)]
    # Now `reshaped_args` contains NumPy arrays of the same shape, suitable for further processing
    result = FeedForwardNN(reshaped_args, X_train)
    y_train_encoded = np.array([y_train])  # np.eye(2)[y_train]
    return binary_cross_entropy(y_train_encoded, result)

import numpy as np

class CustomTask(TaskBase):
    def __init__(self, pars, layer_sizes):
        self.layer_sizes = layer_sizes
        self.pars = pars 

    @property
    def name(self) -> str:
        return 'Custom Task'
    
    def evaluate(self, x: np.ndarray) -> Tuple[pd.DataFrame ,np.ndarray]:
        y = np.zeros((len(x), 1))
        for ind in range(len(x)):
            x_ind = x.iloc[ind].to_numpy()  # Convert Series to NumPy array
            print("Evaluating with parameters:")
            #print("x_ind:", x_ind)
            print('xind shape' , x_ind.shape)
            y[ind] = black_box_function(self.pars, layer_sizes = self.layer_sizes)
        return  x, y

    
    def get_search_space_params(self) -> List[Dict[str, Any]]:
        params = []
        start_idx = 0
        for layer, size in enumerate(self.layer_sizes[:-1]):
            for i in range(size):
                for j in range(self.layer_sizes[layer + 1]):
                    params.append({'name': f'W{layer}_{i}{j}', 'type': 'num', 'lb': -1, 'ub': 1})
            for i in range(self.layer_sizes[layer + 1]):
                params.append({'name': f'b{layer}_{i}', 'type': 'num', 'lb': -1, 'ub': 1})
        return params
        

    def get_parameter_names(self) -> List[str]:
        params = []
        for layer, size in enumerate(self.layer_sizes[:-1]):
            for i in range(size):
                for j in range(self.layer_sizes[layer + 1]):
                    params.append(f'W{layer}_{i}{j}')
            for i in range(self.layer_sizes[layer + 1]):
                params.append(f'b{layer}_{i}')
        return params

# Creating task and optimization
task = CustomTask(parameters, layer_sizes)
searchspace = task.get_search_space()

epochs = 100
bo_builder = BoBuilder(model_id='gp_rd', acq_opt_id='is', acq_func_id='ei', tr_id='basic')
opt = bo_builder.build_bo(search_space=task.get_search_space(), n_init=200)


# Optimization loop
budget_eval = epochs
# Inside the optimization loop
weights_list = []
for _ in range(budget_eval):
    x_next = opt.suggest()
    x_next_reshaped = np.array(x_next).reshape(1, -1)  # Reshape x_next to have shape (1, 25)
    x_next_df = pd.DataFrame(x_next_reshaped, columns=task.get_parameter_names())
    
    y_next = task.evaluate(x_next_df)[1]
    weights_list.append(task.evaluate(x_next_df)[0])
    
   
    opt.observe(x_next, y_next)
   

    
# Printing best result
print('Best parameters:', opt.best_x)
print('Best loss:', opt.best_y)

# Plotting version
fig, ax = plt.subplots(figsize=(7, 5))
y = opt.data_buffer.y.numpy()
regret_y = np.min(np.cumsum(y, axis=1), axis=0)
#plt.plot(np.arange(1,  budget_eval ), regret_y)
#plt.show()

(891, 12)
(714, 12)
['W0' 'b0' 'W1' 'b1' 'W2' 'b2']
Evaluating with parameters:
xind shape (47,)
[array(['W0', 'b0', 'W1', 'b1', 'W2', 'b2'], dtype='<U2')] arr args


ValueError: cannot reshape array of size 6 into shape (4,5)

In [111]:
from BO_methods import CustomTask  # Import CustomTask from BO_methods module
from config_NN_methods import flex_NN, ActivationFunctions
from BO_pipeline import WeightAndBiasOptimizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from mcbo.optimizers.bo_builder import BoBuilder
import numpy as np

# Define the Rastrigin function in 4 dimensions
def rastrigin_4d(x, y, z, w, A=10):
    return (4 * A + (x**2 - A * np.cos(2 * np.pi * x)) + 
            (y**2 - A * np.cos(2 * np.pi * y)) +
            (z**2 - A * np.cos(2 * np.pi * z)) +
            (w**2 - A * np.cos(2 * np.pi * w)))

# Desired dataset length
desired_length = 300

# Calculate the number of points for each dimension
num_points_per_dimension = int(np.sqrt(desired_length))

# Generate X coordinates with the calculated number of points
x = np.linspace(-5.12, 5.12, num_points_per_dimension)
y = np.linspace(-5.12, 5.12, num_points_per_dimension)
z = np.linspace(-5.12, 5.12, num_points_per_dimension)
w = np.linspace(-5.12, 5.12, num_points_per_dimension)
X, Y, Z, W = np.meshgrid(x, y, z, w, indexing='ij')
Z_rastrigin = rastrigin_4d(X, Y, Z, W)

# Flatten X, Y, Z, and W
X = np.column_stack((X.flatten(), Y.flatten(), Z.flatten(), W.flatten()))

# Define a binary threshold for target variable
threshold = np.mean(Z_rastrigin)  # You can adjust the threshold as needed

# Generate binary target variable based on the threshold
y = (Z_rastrigin.flatten() > threshold).astype(int)

# Display the shapes of X and y
print("Shape of X:", X.shape)
print("Shape of y_binary:", y.shape)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


# Usage:
layer_sizes = [4,5,3,1]   # Define your layer sizes

flex_NN_obj = flex_NN(layer_sizes, X_train, y_train)
activation_obj = ActivationFunctions()

parameters = flex_NN_obj.label_parameters()

hidden_activation = activation_obj.relu
output_activation = activation_obj.sigmoid


task = CustomTask(layer_sizes, X_train, y_train)  # Instantiate CustomTask from BO_methods module
searchspace = task.get_search_space()

optimizer_builder = BoBuilder(model_id='gp_rd', acq_opt_id='is', acq_func_id='ei', tr_id='basic')
opt = optimizer_builder.build_bo(search_space=searchspace, n_init=300)
print('opt ========', opt)

weight_bias_optimizer = WeightAndBiasOptimizer(task=task, optimizer=opt)

best_results = weight_bias_optimizer.find_best_weights_and_biases(
                                layer_sizes, 30, X_train, y_train)
print(best_results, 'best res')

Shape of X: (83521, 4)
Shape of y_binary: (83521,)
opt ======== <mcbo.optimizers.bo_base.BoBase object at 0x0000020811A953A0>
{'W0': array([[ 0.75386303, -0.2496896 ,  0.73221999, -0.96464214],
       [-0.90890688,  0.70570393, -0.51083313, -0.65875962],
       [ 0.46190996,  0.56037828, -0.97837064,  0.94476942],
       [-0.89333488, -0.24443206, -0.48435411, -0.21433241],
       [-0.42928374,  0.22868709,  0.26456907, -0.25346136]]), 'b0': array([[ 0.81484518],
       [-0.23393458],
       [ 0.53068482],
       [-0.72632915],
       [-0.55182547]]), 'W1': array([[ 0.30010147, -0.63604584, -0.06168803, -0.71130367, -0.22645877],
       [ 0.62220536,  0.71628871, -0.07101131,  0.45538753, -0.58250687],
       [ 0.23595169,  0.27362234, -0.02473506,  0.27283095,  0.69114687]]), 'b1': array([[-0.5069339 ],
       [ 0.84502607],
       [-0.85363567]]), 'W2': array([[-0.68824404, -0.91556399, -0.70506563]]), 'b2': array([[0.29255867]])}
{'W0': array([[-0.79522228,  0.57259586, -0.04089277,

{'W0': array([[ 0.28139746, -0.85224205, -0.45801509, -0.36845002],
       [-0.30509529,  0.8588615 , -0.70539895, -0.78156623],
       [-0.26850169, -0.21112793, -0.70385756, -0.78374858],
       [-0.95826666, -0.56698062,  0.65739341, -0.72203271],
       [ 0.31859047,  0.8116534 , -0.49601571,  0.33065221]]), 'b0': array([[-0.3303978 ],
       [ 0.30502542],
       [ 0.89751557],
       [ 0.11834558],
       [ 0.49369448]]), 'W1': array([[-0.38131221, -0.10746279, -0.94261587, -0.40702587,  0.91978913],
       [-0.51052542, -0.62310619,  0.92412975,  0.48865256,  0.0206053 ],
       [ 0.91896739, -0.49150945,  0.08011694, -0.65874742,  0.35158978]]), 'b1': array([[ 0.96238434],
       [-0.77173773],
       [ 0.02257835]]), 'W2': array([[0.29925691, 0.38002455, 0.78695538]]), 'b2': array([[-0.18744118]])}
{'W0': array([[ 0.87618539,  0.17075292, -0.51460454,  0.66637141],
       [ 0.68145011,  0.17952311,  0.56134824,  0.84132694],
       [-0.87490857, -0.38974116,  0.78442702,  0.82

{'W0': array([[-0.31742505,  0.19030954,  0.54906787, -0.10814022],
       [ 0.13239617, -0.02204523,  0.89199431, -0.64041182],
       [-0.5743245 , -0.417679  ,  0.23937154, -0.35947256],
       [-0.75882796, -0.41922082, -0.51663949,  0.6230304 ],
       [ 0.30339093,  0.97261269,  0.29305332,  0.14665797]]), 'b0': array([[-0.4604449 ],
       [ 0.80074941],
       [ 0.17104602],
       [ 0.55163501],
       [ 0.25895921]]), 'W1': array([[-0.69367001,  0.0014289 , -0.70783176,  0.84418811, -0.42572181],
       [-0.20013239,  0.41998385, -0.17210428,  0.93355177, -0.1255241 ],
       [-0.35455912,  0.51121908,  0.44456802, -0.0749294 , -0.84979788]]), 'b1': array([[0.50927666],
       [0.34194342],
       [0.23347517]]), 'W2': array([[ 0.15469558, -0.79187378, -0.37153225]]), 'b2': array([[-0.19673166]])}
{'W0': array([[ 0.19319916,  0.6861412 ,  0.43184422, -0.70233428],
       [ 0.97426809,  0.52278163,  0.08109323,  0.65411464],
       [ 0.1531414 ,  0.91857194, -0.96602355, -0.53

In [112]:
len(y)

83521

In [110]:
np.sqrt(300)

17.320508075688775